In [1]:
import pandas as pd
import numpy as np
import scipy as sp

from itertools import chain

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#read in v3 data which is a dictionary of aa and what do they stand for
v3 =pd.read_csv('~/Documents/persona_project/original data and supplements/v3_segments.csv')

In [3]:
#read in number of visit by month
#num_month =pd.read_csv('~/Documents/persona_project/result_num_visit.csv')
#num_month.columns=['DEVICE_ID_HASHED','jan_visit','feb_visit','mar_visit','apr_visit','may_visit', 'june_visit','july_visit','august_visit']
#num_month=num_month.fillna(0)

In [4]:
#find segment ID basic information
#aa=v3.SEGMENT_ID.unique()
#round(v3.SEGMENT_ID.describe(),2)

In [5]:
#read in persona data
persona =pd.read_csv('~/Documents/persona_project/persona_exploration.csv')

In [6]:
persona.shape

(58591, 7)

In [7]:
len(persona.DEVICE_ID_HASHED.drop_duplicates())

58587

In [8]:
#only AA in persona data
aa_persona=persona.AA

In [9]:
#split AA by comma
aa_persona_a=[]
for i in range(len(aa_persona)):
    aa_persona_a.append(aa_persona[i].split(","))

In [10]:
#unlist AA
aa_persona_b=list(chain(*aa_persona_a))

#change AA from list to array
aa_persona_c=np.asarray(aa_persona_b)

#remove missing (nothing is gone, but just make sure)
#aa_persona_all_2 = aa_persona_all_2[~pd.isnull(aa_persona_all_2)]

#put AA into a table 
aa_df=pd.DataFrame(aa_persona_c)
aa_df=aa_df.rename({0: 'aa'}, axis='columns')

In [11]:
#merge aa with segment info

#change aa
aa_df.aa=aa_df.aa.astype('int64')


aa_df1=aa_df.merge(v3, left_on='aa', right_on='SEGMENT_ID', how='left')

In [12]:
aa_df1.SEGMENT_ID=aa_df1.SEGMENT_ID.astype('str')
aa_df1['name']=aa_df1['CATEGORY2']+" "+aa_df1['CATEGORY3'].astype('str')+' '+aa_df1['SEGMENT_NAME']

In [13]:
aa_name=aa_df1['name'].dropna()

In [14]:
#aa_name=' '.join(map(str, aa_df1['name']))

In [17]:
from sklearn.feature_extraction import text
from sklearn.decomposition import LatentDirichletAllocation

vect = text.TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=50000,             # max number of uniq words
                            )


In [18]:
dat = vect.fit_transform(aa_name)

In [ ]:
#print(vect.inverse_transform(dat)[0])

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, learning_method="batch")
X_lda = lda.fit_transform(dat)

In [ ]:
strings=pd.DataFrame(' ', index=persona.DEVICE_ID_HASHED,  columns=['words'])
for j in range(persona.shape[0]):
    for i in range(v3.shape[0]):
        if v3.SEGMENT_ID[i] in persona.AA[j]:
            strings.iloc[j]=strings.iloc[j]+' '+v3.SEGMENT_NAME[i]

In [ ]:
#take a subset of data that has AA larger than or equal to certain frequency (here top 10% frequency)
#aa_sub90=aa_freq2[aa_freq2['freq']>=7347]
#aa_sub90.iloc[101:150,:]
#find the unique higher category information
#aa_sub90.CATEGORY3.unique()

In [ ]:
aa_sub99=aa_freq2[aa_freq2['freq']>=4687]
aa_sub99['cat']=aa_sub99.CATEGORY2.astype('str')+' '+aa_sub99.CATEGORY3.astype('str')+' '+aa_sub99.SEGMENT_NAME.astype('str')

In [ ]:
#dataframe with segment IDs and their associated names/cats
aa_sub99b=aa_sub99.loc[:,['SEGMENT_ID', 'cat']]

In [ ]:
aa_sub99b.to_csv('names.csv')

In [ ]:
#find selected aa in persona table (can't merge, because aa in persona is in long string)
#(group a has these top common words and group b does not have these top common words)

In [ ]:
#create empty new dataframe indicating if device has each of the most common features or not
id=aa_sub99b.SEGMENT_ID.astype('str')
id_str='ID_'+aa_sub99b.SEGMENT_ID.astype('str')
ind=pd.DataFrame(0, index=persona.DEVICE_ID_HASHED, columns=id_str)

In [ ]:
#fill in the ind table. if id in 
for j in range(persona.shape[0]):
    for i in range(len(id)):
        if id[i] in persona.AA[j]:
            ind.iat[j,i]=1
        else:
            ind.iat[j,i]=0

In [ ]:
#merge origina persona data with new AA id indicators 
persona1=persona.merge(ind, left_on='DEVICE_ID_HASHED', right_on='DEVICE_ID_HASHED')

In [ ]:
persona1.to_csv('persona1_processed.csv')

In [ ]:
#persona1.iloc[0:100,:]

In [ ]:
#only focus on the features from AA, and forget about the number of visit per month for now. 

#join with number of visits per month per device 
#persona2=persona1.merge(num_month,left_on='DEVICE_ID_HASHED', right_on='DEVICE_ID_HASHED')

#because the data are not the whole samle, the device ID do not match in aa table and num_visit table. 
#take inner join to make the dataset smaller, and easier to work with. 
#will change after able to work with the whole dataset 

#persona2.shape

In [ ]:
#persona2b=persona2.iloc[:,2:]

In [ ]:
#persona2b.index=persona2.DEVICE_ID_HASHED

In [ ]:
#persona2c=persona2b.drop(columns=['SEGMENT_ID'])
#persona2c=persona2c.drop(columns=['INCOME_RANGE'])
#persona2c=persona2c.drop(columns=['AGE_RANGE'])